# some functions for LLM completetions

In [26]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from zhipuai import ZhipuAI
from http import HTTPStatus
from dashscope import Generation

In [27]:
import dashscope
# 加载 .env 文件
load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
glmapikey = os.getenv('glmapikey')
ali_key = os.getenv("ALI_API_KEY")
moonshot_key = os.getenv("MOONSHOT_API_KEY")

In [28]:
# 如果你没有将api key放在环境变量中，可以直接作为字符串赋值给变量
DEEPSEEK_API_KEY = "your DEEPSEEK_API_KEY"
glmapikey = "143e7d9483a69850172f2ec5978375a0.MIlVf8wYW5w8bdVg"
ali_key = "your ali_key"
moonshot_key = "sk-pdsgUuwGtxpKt3kmaJJ17EWbsYbfkamsk2w4yOqsPPPn8A5b"

In [29]:
def llm_kimi(prompt, temperature=0.5):
    from openai import OpenAI

    client = OpenAI(
        api_key=moonshot_key,
        base_url="https://api.moonshot.cn/v1",
    )

    completion = client.chat.completions.create(
      model="moonshot-v1-8k",
      messages=[
            {"role": "user", "content": prompt}
      ],
      temperature=temperature
    )
    return completion.choices[0].message.content

In [30]:
def llm_gpt(prompt, temperature = 0.5):
    from openai import OpenAI
    client = OpenAI()

    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {
          "role": "user",
          "content": prompt
        }
      ],
      temperature=temperature,
      max_tokens=500,
      top_p=1
    )
    content = response.choices[0].message.content
    return content

In [31]:
def llm_deepseek(prompt, temperature=0.5):
    # for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
    client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-coder",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
      ],
        max_tokens=1024,
        temperature=temperature,
        stream=False
    )

    return response.choices[0].message.content

In [32]:
def llm_zhipuai_completion(prompt, temperature=0.5):
    client = ZhipuAI(api_key=glmapikey) 
    response = client.chat.completions.create(
        model="glm-4",  
        messages=[
                  {"role": "user", "content": prompt}
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content

In [33]:
def ali_llama_chat(prompt, temperature=0.5):
    """
    使用阿里云的llama模型进行对话
    """
    dashscope.api_key = ali_key
    model = "llama3-70b-instruct"
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}]
    response = Generation.call(model=model,
                               messages=messages,
                               temperature = temperature,
                               max_tokens = 500,
                               result_format='message')
    if response.status_code == HTTPStatus.OK:
        generated_text = response.output.choices[0].message.content
        # 返回生成的文本
        return generated_text
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))

# self-consistency CoT

In [34]:
prompt = """
Email to be Classified:

Hello,

I have discovered a major security vulnerability in your system. Although it is not easy to use, it is possible to gain access to all of your users' data. I have attached a proof of concept. Please fix this issue as soon as possible.

Cheers,

Donny

Task: Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company.

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company. Let's think step by step.

Output your final classification ouput in json format with key "classification" and values: IMPORTANT or NOT IMPORTANT
"""

In [35]:
print(prompt)


Email to be Classified:

Hello,

I have discovered a major security vulnerability in your system. Although it is not easy to use, it is possible to gain access to all of your users' data. I have attached a proof of concept. Please fix this issue as soon as possible.

Cheers,

Donny

Task: Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company.

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company. Let's think step by step.

Output your final classification ouput in json format with key "classification" and values: IMPORTANT or NOT IMPORTANT



In [36]:
r1 = llm_zhipuai_completion(prompt=prompt, temperature=0.99)

In [37]:
r2 = llm_zhipuai_completion(prompt=prompt, temperature=0.99)

In [38]:
r3 = llm_zhipuai_completion(prompt=prompt, temperature=0.99)

In [39]:
print(r3)

```json
{
  "classification": "IMPORTANT"
}
```>

This email should be classified as IMPORTANT for a software company, as it reports a major security vulnerability that could potentially affect all users' data. Addressing such a vulnerability quickly is critical for maintaining the security and trust of the system's users.


In [40]:
import re
import json

def extract_json_from_text(text):
    # Regex pattern to find JSON-like structures
    json_pattern = re.compile(r'\{(?:[^{}]*|\"(?:\\.|[^"\\])*\")*?\}')
    
    # Find all potential JSON strings
    matches = json_pattern.findall(text)
    
    json_objects = []
    
    for match in matches:
        try:
            # Try to parse the matched string as JSON
            json_object = json.loads(match)
            json_objects.append(json_object)
        except json.JSONDecodeError:
            # If parsing fails, it's not valid JSON
            continue
    
    return json_objects

# Example usage
text = """
Here is some text that might contain JSON:
Some irrelevant text...
{"classification": "IMPORTANT"}
More irrelevant text...
{"valid": true, "count": 10}
"""

extracted_json = extract_json_from_text(text)
print(extracted_json)


[{'classification': 'IMPORTANT'}, {'valid': True, 'count': 10}]


In [41]:
extracted_json = extract_json_from_text(r3)

In [42]:
extracted_json[0]["classification"]

'IMPORTANT'

In [43]:
from collections import Counter

def find_most_frequent_value_dictionaries(dict_list):
    if not dict_list:
        return None  # 如果列表为空，返回None

    # 提取所有字典中的value到一个新列表中
    values = [dict_[next(iter(dict_))] for dict_ in dict_list]
    
    # 使用Counter统计每个value的出现次数
    value_counts = Counter(values)
    
    # 找到出现次数最多的value
    if not value_counts:
        return None  # 如果没有value，返回None
    most_frequent_value, _ = value_counts.most_common(1)[0]
    return most_frequent_value

In [44]:
def find_mean_value_dictionaries(dict_list):
    if not dict_list:
        return None  # 如果列表为空，返回None

    # 提取所有字典中的value到一个新列表中
    values = [dict_[next(iter(dict_))] for dict_ in dict_list]
    
    value = sum(values) / len(values) if values else 0
    return value

In [45]:
def SC_CoT(prompt, routes_count, model_type="classification"):
    results=[]
    for i in range(routes_count):
        response = llm_zhipuai_completion(prompt=prompt, temperature=0.99)
        if len(extract_json_from_text(response)) > 0:
            extracted_response = extract_json_from_text(response)[0]
            results.append(extracted_response)
    if model_type=="classification":
        value = find_most_frequent_value_dictionaries(results)
    else:
        value = find_mean_value_dictionaries(results)
    return results, value

In [46]:
results, most_frequent_value = SC_CoT(prompt=prompt, routes_count=3)

In [47]:
results, most_frequent_value

([{'classification': 'IMPORTANT'},
  {'classification': 'IMPORTANT'},
  {'classification': 'IMPORTANT'}],
 'IMPORTANT')

# apply the self-consistency CoT in stock price movement prediction

In [48]:
# load the data
import pandas as pd

In [49]:
data = pd.read_csv("train1.csv")

In [50]:
data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [51]:
#特征工程
df = data.drop(columns=['loan_status'])

In [52]:
df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123,PERSONAL,D,35000,16.02,0.59,Y,3
1,21,9600,OWN,5,EDUCATION,B,1000,11.14,0.10,N,2
2,25,9600,MORTGAGE,1,MEDICAL,C,5500,12.87,0.57,N,3
3,23,65500,RENT,4,MEDICAL,C,35000,15.23,0.53,N,2
4,24,54400,RENT,8,MEDICAL,C,35000,14.27,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...
94,23,62500,RENT,7,MEDICAL,B,26000,11.71,0.42,N,2
95,23,120000,RENT,1,EDUCATION,B,25600,12.69,0.21,N,3
96,24,12360,OWN,2,MEDICAL,C,1600,13.57,0.13,N,3
97,22,60000,RENT,0,VENTURE,B,25475,10.99,0.42,N,3


In [53]:
#df = df.sort_values(by=["trade_date"])

In [54]:
df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123,PERSONAL,D,35000,16.02,0.59,Y,3
1,21,9600,OWN,5,EDUCATION,B,1000,11.14,0.10,N,2
2,25,9600,MORTGAGE,1,MEDICAL,C,5500,12.87,0.57,N,3
3,23,65500,RENT,4,MEDICAL,C,35000,15.23,0.53,N,2
4,24,54400,RENT,8,MEDICAL,C,35000,14.27,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...
94,23,62500,RENT,7,MEDICAL,B,26000,11.71,0.42,N,2
95,23,120000,RENT,1,EDUCATION,B,25600,12.69,0.21,N,3
96,24,12360,OWN,2,MEDICAL,C,1600,13.57,0.13,N,3
97,22,60000,RENT,0,VENTURE,B,25475,10.99,0.42,N,3


In [55]:
import json

In [56]:
# Convert dataframe to JSON
df_json = df.to_json(orient="records")

In [57]:
df_json

'[{"person_age":22,"person_income":59000,"person_home_ownership":"RENT","person_emp_length":123,"loan_intent":"PERSONAL","loan_grade":"D","loan_amnt":35000,"loan_int_rate":16.02,"loan_percent_income":0.59,"cb_person_default_on_file":"Y","cb_person_cred_hist_length":3},{"person_age":21,"person_income":9600,"person_home_ownership":"OWN","person_emp_length":5,"loan_intent":"EDUCATION","loan_grade":"B","loan_amnt":1000,"loan_int_rate":11.14,"loan_percent_income":0.1,"cb_person_default_on_file":"N","cb_person_cred_hist_length":2},{"person_age":25,"person_income":9600,"person_home_ownership":"MORTGAGE","person_emp_length":1,"loan_intent":"MEDICAL","loan_grade":"C","loan_amnt":5500,"loan_int_rate":12.87,"loan_percent_income":0.57,"cb_person_default_on_file":"N","cb_person_cred_hist_length":3},{"person_age":23,"person_income":65500,"person_home_ownership":"RENT","person_emp_length":4,"loan_intent":"MEDICAL","loan_grade":"C","loan_amnt":35000,"loan_int_rate":15.23,"loan_percent_income":0.53,"cb

In [58]:
prompt = f"""Given credit data for a group of users, predict loan_status_value (a value of 1 represents default and a value of 0 represents non-default).

Known feature information in JSON format
# # # {df_json} # # #

Analyze the characteristics of the training set users and give your predictions and reasons based on financial risk control knowledge.
Don't explain the method of machine learning to me, I need you to think of yourself as a financial expert who can directly predict the loan status based on the given data characteristics and give convincing reasons.

Forecast target:
Predicted loan status of unknown tag training set user :[loan_status_value_0/loan_status_value_1]

Output requirements:
- The sum of probabilities of loan_status_value_0 and loan_status_value_1 must be equal to 1.
- Output the existing feature data and predicted loan state probability in JSON format. The keywords and values of the existing feature data are the same as those of the known data. The predicted loan state probability keywords are loan_status_value_0 and loan_status_value_1. (Note: the known feature data should have the same value and format as the previously given)

Now, think about it step by step.
"""


In [59]:
print(prompt)

Given credit data for a group of users, predict loan_status_value (a value of 1 represents default and a value of 0 represents non-default).

Known feature information in JSON format
# # # [{"person_age":22,"person_income":59000,"person_home_ownership":"RENT","person_emp_length":123,"loan_intent":"PERSONAL","loan_grade":"D","loan_amnt":35000,"loan_int_rate":16.02,"loan_percent_income":0.59,"cb_person_default_on_file":"Y","cb_person_cred_hist_length":3},{"person_age":21,"person_income":9600,"person_home_ownership":"OWN","person_emp_length":5,"loan_intent":"EDUCATION","loan_grade":"B","loan_amnt":1000,"loan_int_rate":11.14,"loan_percent_income":0.1,"cb_person_default_on_file":"N","cb_person_cred_hist_length":2},{"person_age":25,"person_income":9600,"person_home_ownership":"MORTGAGE","person_emp_length":1,"loan_intent":"MEDICAL","loan_grade":"C","loan_amnt":5500,"loan_int_rate":12.87,"loan_percent_income":0.57,"cb_person_default_on_file":"N","cb_person_cred_hist_length":3},{"person_age":2

In [60]:
r = llm_zhipuai_completion(prompt=prompt, temperature=0.99)

In [61]:
print(r)

Based on the given feature data, let's analyze the financial risk factors and make predictions for the loan status:

1. The individual has a person_age of 23, which is a relatively younger age. Younger individuals often have less established credit histories and may pose higher credit risk.

2. With a person_income of 500000, the income is very high, suggesting good repayment capacity and lower risk.

3. Being a mortgage owner indicates some level of financial stability, as owning a mortgage suggests responsible financial management.

4. The person_emp_length of 7 months is relatively short, indicating potentially lower job security and higher risk.

5. The loan_intent is debt_consolidation, which is often associated with individuals facing financial difficulties, indicating higher risk.

6. A loan_grade of B implies a moderate credit risk level.

7. The loan_amnt of 30000 is a significant amount, raising the risk level given the borrower's young age and shorter employment length.

8. 

In [62]:
extract_json_from_text(r)

[{'person_age': 23,
  'person_income': 500000,
  'person_home_ownership': 'MORTGAGE',
  'person_emp_length': 7,
  'loan_intent': 'DEBTCONSOLIDATION',
  'loan_grade': 'B',
  'loan_amnt': 30000,
  'loan_int_rate': 10.65,
  'loan_percent_income': 0.06,
  'cb_person_default_on_file': 'Y',
  'cb_person_cred_hist_length': 3},
 {'loan_status_value_0': 0.3, 'loan_status_value_1': 0.7}]

In [63]:
results, majority_prediction = SC_CoT(prompt=prompt, routes_count=3, model_type="regression")

In [66]:
find_mean_value_dictionaries(results)

15.325

In [67]:
results

[{'person_age': 30,
  'person_income': 75000,
  'person_home_ownership': 'RENT',
  'person_emp_length': 5,
  'loan_intent': 'PERSONAL',
  'loan_grade': 'C',
  'loan_amnt': 30000,
  'loan_int_rate': 12.5,
  'loan_percent_income': 0.4,
  'cb_person_default_on_file': 'N',
  'cb_person_cred_hist_length': 3},
 {'loan_status_value_0': 0.65, 'loan_status_value_1': 0.35}]